In [257]:
from gurobipy import *
import gurobipy as gb
import numpy as np
import networkx as nx

class vehicle:
    def __init__(self):
        self.Va = 900
        self.Wa = 450
        self.Ka = 0.12
        self.Qa = 36
        self.Ca = 0.04
        self.Da = 36

class EV(vehicle):
    def __init__(self, scalability):
        super().__init__()
        self.B0 = 0.35
        self.Bmax = 1.0 if scalability else 0.5
        self.omega = 0.33
        self.epsilon = 0.00018

class ICV(vehicle):
    def __init__(self):
        super().__init__()

In [258]:
#def optimisation_model(link_length, paths, T, timestep, scalability, budget):
link_length = {1: 0, 2: 900, 3: 1800, 4: 900, 5: 3600, 6: 900, 7: 0}
link = list(link_length.keys())
link_source = link[0]
link_sink = link[-1]

paths = [[1, 2, 4, 6, 7], [1, 2, 5, 7], [1, 3, 6, 7]]
edges = [(1, 2), (1, 3), (2, 4), (2, 5), (3, 6), (4, 6), (5, 7), (6, 7)]
# Generate paths from the edges
Grafo = nx.DiGraph()
Grafo.add_edges_from(edges)
paths = list(nx.all_simple_paths(Grafo, source=1, target=7))

# Generate incoming links and outgoing links relative to link a
incoming_links = {a: list(Grafo.predecessors(a)) for a in Grafo.nodes}
# {1: [], 2: [1], 3: [1], 4: [2], 5: [2], 6: [3, 4], 7: [5, 6]}
outgoing_links = {a: list(Grafo.successors(a)) for a in Grafo.nodes}
# {1: [2, 3], 2: [4, 5], 3: [6], 4: [6], 5: [7], 6: [7], 7: []}
T = 60
timestep = 0.25
budget = 1800
scalability = False

ev = EV(scalability)
icv = ICV()
M = ["EV", "ICV"]

time_idx = [round(i * timestep, 6) for i in range(int(T / timestep) + 1)]

model = gb.Model()
model.modelSense = gb.GRB.MAXIMIZE
model.setParam('OutputFlag', 0)
    

In [259]:
print(time_idx)

[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.0, 7.25, 7.5, 7.75, 8.0, 8.25, 8.5, 8.75, 9.0, 9.25, 9.5, 9.75, 10.0, 10.25, 10.5, 10.75, 11.0, 11.25, 11.5, 11.75, 12.0, 12.25, 12.5, 12.75, 13.0, 13.25, 13.5, 13.75, 14.0, 14.25, 14.5, 14.75, 15.0, 15.25, 15.5, 15.75, 16.0, 16.25, 16.5, 16.75, 17.0, 17.25, 17.5, 17.75, 18.0, 18.25, 18.5, 18.75, 19.0, 19.25, 19.5, 19.75, 20.0, 20.25, 20.5, 20.75, 21.0, 21.25, 21.5, 21.75, 22.0, 22.25, 22.5, 22.75, 23.0, 23.25, 23.5, 23.75, 24.0, 24.25, 24.5, 24.75, 25.0, 25.25, 25.5, 25.75, 26.0, 26.25, 26.5, 26.75, 27.0, 27.25, 27.5, 27.75, 28.0, 28.25, 28.5, 28.75, 29.0, 29.25, 29.5, 29.75, 30.0, 30.25, 30.5, 30.75, 31.0, 31.25, 31.5, 31.75, 32.0, 32.25, 32.5, 32.75, 33.0, 33.25, 33.5, 33.75, 34.0, 34.25, 34.5, 34.75, 35.0, 35.25, 35.5, 35.75, 36.0, 36.25, 36.5, 36.75, 37.0, 37.25, 37.5, 37.75, 38.0, 38.25, 38.5, 38.75, 39.0, 39.25, 39.5, 39.75,

In [260]:
x = model.addVars(link, vtype=gb.GRB.BINARY, name="x")
y = model.addVars(M, len(paths), vtype=gb.GRB.BINARY, name="y")
B = model.addVars(link, len(paths), lb=-GRB.INFINITY, ub=ev.Bmax, name="B") #lb di B a meno infinito

n = model.addVars(link, M, time_idx, lb=0, ub=ev.Ka * 3600, name="n")
u = model.addVars(link, M, time_idx, lb=0, ub=ev.Qa, name="u")
v = model.addVars(link, M, time_idx, lb=0, ub=ev.Qa, name="v")
f = model.addVars(link, link, M, time_idx, lb=0, ub=ev.Qa, name="f")

In [261]:
print(link_length[5])

3600


In [262]:
M_big=2.0
model.addConstr(gb.quicksum(link_length[a] * x[a] for a in link[1:-1]) <= budget)

G = model.addVars(link, len(paths), vtype=GRB.BINARY, name="G")
for p in range(len(paths)):
    model.addConstr(B[link_source, p] == ev.B0)
    for i in range(1, len(paths[p])):
        b = paths[p][i-1]
        a = paths[p][i]
        energy = B[b, p] - (ev.epsilon * link_length[a]) + (ev.omega * (link_length[a] / ev.Va) * x[a])
        model.addConstr(B[a, p] >=  energy - M_big * G[a, p])
        model.addConstr(B[a, p] >= ev.Bmax - M_big * (1 - G[a, p]))
        model.addConstr(B[a, p] <= energy)
        model.addConstr(B[a, p] <= ev.Bmax)

        #model.addConstr(energy-ev.Bmax>=-M_big*G[a,p])
    for a in paths[p]:
        model.addConstr(B[a, p] <= ev.Bmax + M_big * (1 - y["EV", p]))
        model.addConstr(B[a, p] >= 0 - M_big * (1 - y["EV", p]))
    model.addConstr(y["ICV", p] == 1) #tutti i path per icv sono fattibili

In [263]:
deltaKron = {(p, a): 1 if a in paths[p] else 0 for p in range(len(paths)) for a in link}
print(deltaKron)
alpha = model.addVars(link, M, time_idx, lb=0.0, ub=1.0, name="alpha")
for a in link:
    for t in time_idx:
        model.addConstr(gb.quicksum(alpha[a, m, t] for m in M) == 1)

alpha_EV = gb.quicksum(u[link_source, "EV", t] for t in time_idx)
alpha_ICV = gb.quicksum(u[link_source, "ICV", t] for t in time_idx)
den_alpha = alpha_EV + alpha_ICV + 1e-5

#model.addConstr(gb.quicksum(alpha[link_source, "EV", t] for t in time_idx) * den_alpha == alpha_EV)
#model.addConstr(gb.quicksum(alpha[link_source, "ICV", t] for t in time_idx) * den_alpha == alpha_ICV)

{(0, 1): 1, (0, 2): 1, (0, 3): 0, (0, 4): 1, (0, 5): 0, (0, 6): 1, (0, 7): 1, (1, 1): 1, (1, 2): 1, (1, 3): 0, (1, 4): 0, (1, 5): 1, (1, 6): 0, (1, 7): 1, (2, 1): 1, (2, 2): 0, (2, 3): 1, (2, 4): 0, (2, 5): 0, (2, 6): 1, (2, 7): 1}


In [264]:
for a in link:
     for t in time_idx:
        for m in M:
            model.addConstr(n[a, m, t] == gb.quicksum(u[a, m, k] - v[a, m, k] for k in time_idx if k <= t))
        if a != link_source and a != link_sink:
            t_in = max(0, round(t - link_length[a] / ev.Va , 6))
            t_out = max(0, round(t - link_length[a] / ev.Wa , 6))
            for m in M:
                model.addConstr(gb.quicksum(u[a, m, k] for k in time_idx if t_in <= k <= t) <= n[a, m, t])
                model.addConstr(n[a, m, t] + gb.quicksum(v[a, m, k] for k in time_idx if t_out <= k <= t) <= ev.Ka * link_length[a] * alpha[a, m, t])
                model.addConstr(gb.quicksum(f[b, a, m, t] for b in link) <= ev.Qa * gb.quicksum(deltaKron[p, a] * y[m, p] for p in range(len(paths))))

        if a != link_source:
                for m in M:
                    model.addConstr(u[a, m, t] == gb.quicksum(f[b, a, m, t] for b in link))
        if a != link_sink:
                for m in M:
                    model.addConstr(v[a, m, t] == gb.quicksum(f[a, b, m, t] for b in link))

In [265]:
for t in time_idx:
    for m in M:
        model.addConstr(u[link_source, m, t] == ev.Da)
        model.addConstr(v[link_sink, m, t] == 0)

In [266]:
for t in time_idx:
    for m in M:
        for a in link:
            # Formula 12: conservation of vehicle numbers
            model.addConstr(n[a,m,t] == gb.quicksum(u[a,m,k] - v[a,m,k] for k in time_idx if k <= t))
            if a != link_source and a!= link_sink:
                # formula 13: upstream capacity
                t_in = max(0, round(t - link_length[a]/900 + 1))
                model.addConstr(gb.quicksum(u[a,m,k] for k in time_idx if t_in <= k <= t) <= n[a,m,t])
                # formula 14: downstream capacity
                t_out = max(0, round(t - link_length[a]/450 + 1))
                model.addConstr(n[a,m,t] + gb.quicksum(v[a,m,k] for k in time_idx if t_out <= k <= t) <= 0.12 * link_length[a] * alpha[link_source,m,t])
                # formula 19: flow capacity of EV on links
                # paper does the sum of f[b,a] but with constraint 16 we saw it is =u[a]
                model.addConstr(u[a,m,t] <= 36 * gb.quicksum(deltaKron[p,a] * y[m,p] for p in range(len(paths))))

            if a != link_source:
             # formula 16: flux conservation for incoming vehicles
                model.addConstr(u[a,m,t] == gb.quicksum(f[b,a,m,t] for  b in incoming_links[a]))
            if a != link_sink:
             # formula 17:  flux conservation for outgoing vehicles
                model.addConstr(v[a,m,t] == gb.quicksum(f[a,b,m,t] for  b in outgoing_links[a]))
            
        # formula 15: source link constraint is the demand rate of vehicle M at time step t
        model.addConstr(u[link_source,m,t] == ev.Da)
        
        # formula 18: sink link constraint
        model.addConstr(v[link_sink,m,t] == 0)
        

In [267]:
# Constraints (20)-(23): Supply and Demand constraints for each link
S = model.addVars(link, time_idx, lb=0, name="S")
D = model.addVars(link, time_idx, lb=0, name="D")
for a in link[1:-1]:
    for t in time_idx:
        inflow_s = gb.quicksum(u[a, m, k] for m in M for k in time_idx if k < t)
        outflow_s = gb.quicksum(v[a, m, k] for m in M for k in time_idx if k <= t - link_length[a] / ev.Wa)
        supply_sum = ev.Ka * link_length[a] + outflow_s - inflow_s
        model.addConstr(S[a, t] <= ev.Qa)
        model.addConstr(S[a, t] <= supply_sum)
        model.addConstr(gb.quicksum(u[a, m, t] for m in M) <= S[a, t])

        inflow_d = gb.quicksum(v[a, m, k] for m in M for k in time_idx if k < t)
        outflow_d = gb.quicksum(u[a, m, k] for m in M for k in time_idx if k <= t - link_length[a] / ev.Va)
        demand_sum = outflow_d - inflow_d
        model.addConstr(D[a, t] <= ev.Qa)
        model.addConstr(D[a, t] <= demand_sum)
        model.addConstr(gb.quicksum(v[a, m, t] for m in M) <= D[a, t])

In [268]:
model.setObjective(gb.quicksum(f[b, link_sink, m, t] * (len(time_idx)+1-t) for m in M for t in time_idx for b in incoming_links[link_sink]))

model.optimize()

In [269]:
if model.status == gb.GRB.OPTIMAL or model.status == gb.GRB.SUBOPTIMAL:
    for a in link:
        if x[a].x > 0.5:
            print(f"Link {a} ha WCL installato.")
else:
    print(f"⚠️ Il modello non è stato risolto correttamente. Status: {model.status}")

⚠️ Il modello non è stato risolto correttamente. Status: 3


In [270]:
total_flow_to_sink = model.ObjVal
print(f"\n📄 System outflow: {total_flow_to_sink:.1f}\n")

for a in link:
        print(f"x[{a}] (WCL installata): {x[a].x:.0f}")

for a in link:
        for p in range(len(paths)):
            print(f"B[{a},{p}] = {B[a, p].x:.3f}")



AttributeError: Unable to retrieve attribute 'ObjVal'

In [ ]:
print("\n🔌 Link selezionati per WCL:")
for a in link:
    if x[a].x > 0.5:
        print(f"  - Link {a}")
        
print("\n🚗 Path selezionati per EV:")
for p in range(len(paths)):
    if y["EV", p].X > 0.5:
        print(f"  - Path {paths[p]}")
        
print(f"\n🎯 Valore funzione obiettivo (outflow): {model.ObjVal:.2f}")
        
print("\n🔋 Stato di energia B[a, p]:")
for p in range(len(paths)):
    for a in link:
        if (a, p) in B:
            val = B[a, p].X
            if val > 1e-4:
                print(f"  B[{a}, {p}] = {val:.4f}")
                
print("\n📈 Flusso in entrata e uscita (solo se > 0):")
for a in link:
    for t in np.round(np.arange(0, T + timestep, timestep), 4):
        for m in M:
            if u[a, m, t].X > 1e-3 or v[a, m, t].X > 1e-3:
                print(f"  t={t:.2f}, link={a}, tipo={m}: u = {u[a,m,t].X:.2f}, v = {v[a,m,t].X:.2f}")
                
print("\n↔️ Flusso f[a, b, m, t] (solo se > 0):")
for a in link:
    for b in link:
        for m in M:
            for t in np.round(np.arange(0, T + timestep, timestep), 4):
                if f[a, b, m, t].X > 1e-3:
                    print(f"  f[{a},{b},{m},{t:.2f}] = {f[a,b,m,t].X:.2f}")


🔌 Link selezionati per WCL:
  - Link 3

🚗 Path selezionati per EV:
  - Path [1, 3, 6, 7]

🎯 Valore funzione obiettivo (outflow): 207900.00

🔋 Stato di energia B[a, p]:
  B[1, 0] = 0.3500
  B[2, 0] = 0.1880
  B[3, 0] = 0.5000
  B[4, 0] = 0.0260
  B[5, 0] = 0.5000
  B[1, 1] = 0.3500
  B[2, 1] = 0.1880
  B[3, 1] = 0.5000
  B[4, 1] = 0.5000
  B[6, 1] = 0.5000
  B[1, 2] = 0.3500
  B[2, 2] = 0.5000
  B[3, 2] = 0.5000
  B[4, 2] = 0.5000
  B[5, 2] = 0.5000
  B[6, 2] = 0.3380
  B[7, 2] = 0.3380

📈 Flusso in entrata e uscita (solo se > 0):
  t=0.00, link=1, tipo=EV: u = 36.00, v = 36.00
  t=0.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=0.25, link=1, tipo=EV: u = 36.00, v = 36.00
  t=0.25, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=0.50, link=1, tipo=EV: u = 36.00, v = 36.00
  t=0.50, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=0.75, link=1, tipo=EV: u = 36.00, v = 36.00
  t=0.75, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=1.00, link=1, tipo=EV: u = 36.00, v = 36.00
  t=1.00, link=1, tipo=ICV: